In [ ]:
import cassiopeia as cass
from cassiopeia import Summoner, Match
from cassiopeia.data import Season, Queue
from collections import Counter
from ratelimit import limits, sleep_and_retry
import arrow
import requests
import pandas as pd
import datetime

@sleep_and_retry
@limits(calls=20, period=1)
@limits(calls=100, period=120)
def get_full_match_history(key, summoner):
    data = {
        "X-Riot-Token": key
    }
    
    total_matches = 0
    fetched_records = 1
    full_history = None

    while fetched_records > 0:
        print(f"Fetching records {total_matches} to {total_matches+100}")
        
        query = "https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}".format(puuid=summoner.puuid,
                                                                                                                                  start=total_matches,
                                                                                                                                  count=100)
        
        response = requests.get(query, headers=data)
        
        history = response.content.decode('UTF-8').replace('"',"").strip("[]").split(",")
        
        history = [ele for ele in history if ele != ""]

        if full_history is None:
            full_history = history
        else:
            full_history.extend(history)
            
        fetched_records = len(history)
        total_matches = total_matches + fetched_records

    return full_history

def start_error(row):
    try:
        return row.match.creation#pd.to_datetime(row.match.creation.datetime, utc=True)  
    except:
        return 0#return arrow.get('1980-01-01')#datetime(1980,1,1,0,0,0).replace(tzinfo=timezone.utc)
    
def duration_error(row):
    try:
        return row.match.duration
    except:
        return 0
    
def result(row):
    return {'name': row.summoner.name,  'result': row.stats.win}

def get_wins(match, name):
    try:
        results = pd.DataFrame(map(result, match.participants)).drop_duplicates()
        if results['result'][results['name'] == name].values[0] == True:
            return 'Win'
        elif results['result'][results['name'] == name].values[0] == False:
            return 'Loss'
        else:
            return 'Other'
    except:
        return 'Other'

def add_times(start, duration):
    return start + duration

def create_player_stats(key, player, role):
    cass.set_riot_api_key(key)

    summoner = Summoner(name=player, region="NA")

    match_history = get_full_match_history(key, summoner)

    print(f'Finished pulling {summoner.name}, they had {len(match_history)} games on this account')

    games = pd.DataFrame()
    games['match_id'] = match_history
    games['match'] = games.apply(lambda row: Match(id=row.match_id, region='NA'), axis = 1)
    games['start_time'] = games.apply(lambda row: start_error(row), axis = 1)
    games['duration'] = games.apply(lambda row: duration_error(row), axis = 1)
    games['end_time'] = games.apply(lambda row: add_times(row['start_time'], row['duration']), axis = 1)
    games = games.loc[(games['start_time'] != 0) & (games['duration'] != 0) & (games['end_time'] != 0)]
    games = games.loc[games['start_time'] >= arrow.get('2022-01-01')]
    games['player'] = summoner.name
    games['results'] = games.apply(lambda x: get_wins(x['match'], x['player']), axis = 1)
    games['start_next'] = games.groupby('player').start_time.shift(1)

    play_sessions = [1]
    session_count = 1

    for i in range(1, len(games)):
        if (games['start_next'][i] - games['end_time'][i]) > datetime.timedelta(seconds = 1800):
            session_count += 1
        play_sessions.append(session_count)

    games['sessions'] = play_sessions

    grp = games.groupby('sessions')
    grp_out = grp.agg(win_count = ('results', lambda x: (x=='Win').sum()), loss_count = ('results', lambda x: (x=='Loss').sum()), session_start = ('start_time', min), session_end = ('end_time', max))
    grp_out.reset_index(inplace=True)
    grp_out['next_session'] = grp_out['sessions'] - 1
    grp_out['start_next_session'] = grp_out.merge(grp_out, left_on="next_session", right_on="sessions", how='left')['session_start_y']
    grp_out['total_count'] = grp_out['win_count'] + grp_out['loss_count']
    grp_out['win_ratio'] = grp_out['win_count'] / grp_out['total_count']
    grp_out['session_length'] = grp_out['session_end'] - grp_out['session_start']
    grp_out['time_until_next_session'] = grp_out['start_next_session'] - grp_out['session_end']

    stats = {'player': games['player'][0], 'role': role, 'avg_games_per_session': grp_out['total_count'].mean(), 'avg_length_of_session': grp_out['session_length'].mean(), \
             'avg_length_of_winning_session': grp_out['session_length'].loc[grp_out['win_ratio'] > 0.5].mean(), \
             'avg_length_of_losing_session': grp_out['session_length'].loc[grp_out['win_ratio'] < 0.5].mean(), \
             'avg_length_of_even_session': grp_out['session_length'].loc[grp_out['win_ratio'] == 0.5].mean(), 'avg_length_of_break': grp_out['time_until_next_session'].mean(), \
             'avg_length_of_break_after_winning': grp_out['time_until_next_session'].loc[grp_out['win_ratio'] > 0.5].mean(), \
             'avg_length_of_break_after_losing': grp_out['time_until_next_session'].loc[grp_out['win_ratio'] < 0.5].mean(), \
             'avg_length_of_break_after_even': grp_out['time_until_next_session'].loc[grp_out['win_ratio'] == 0.5].mean()}
    
    return stats


players = ['Nebs97', 'djruhnke', 'Elend Venture', 'EnderAWiggin', 'lentzf', 'GiveEmSomePepper', 'BoatRaceGod', 'NatGlanzman', 'shakaynery', 'LordZooplankton']
pros = ['blaberfish2', 'TF Blade', 'Doublelift', 'Smoothie', 'Ssumdayday', 'Meteos', 'Ablazeolive', 'Imaqtpie', 'Shiphtur', 'AnDa']
combined_stats = pd.DataFrame()

for p in players:
    stats = create_player_stats('RGAPI-38abfb73-273e-4606-b82d-7ec25c72228c', p, 'non_pro')
    player_stats = pd.DataFrame(data=stats, index=[0])
    combined_stats = pd.concat([combined_stats, player_stats], axis = 0, sort = False).reset_index(drop = True)

for p in pros:
    stats = create_player_stats('RGAPI-38abfb73-273e-4606-b82d-7ec25c72228c', p, 'pro')
    player_stats = pd.DataFrame(data=stats, index=[0])
    combined_stats = pd.concat([combined_stats, player_stats], axis = 0, sort = False).reset_index(drop = True)
        

Fetching records 0 to 100
Fetching records 100 to 200
Fetching records 200 to 300
Fetching records 300 to 400
Fetching records 400 to 500
Fetching records 500 to 600
Fetching records 583 to 683
Finished pulling Nebs97, they had 583 games on this account
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4355408942
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4355391932
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_3938060270
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_3930155743
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_3929996307
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_3929956507
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_3938060270
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_3930155743
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_3

In [ ]:
combined_stats

In [154]:
import plotly.graph_objects as go

# df = pd.DataFrame([
#     dict(Task="Job A", Start='2009-01-01', Finish='2009-02-28'),
#     dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15'),
#     dict(Task="Job C", Start='2009-02-20', Finish='2009-05-30')
# ])

fig = go.timeline(games, x_start="start_time", x_end="end_time", y="player", color="player", color_discrete_sequence = ['red'], opacity=1)
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.update_xaxes(range=['2022-01-20 18:15:29+00:00', '2022-06-20 18:15:29+00:00'])
fig.show()

AttributeError: module 'plotly.graph_objects' has no attribute 'timeline'

In [156]:
# Average games/session
# Average play session length
# Average play session length on losing day
# Average play session length on winning day